In [24]:
#Imports
%matplotlib inline
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

In [25]:
#Network parameters
batch_size = 64
img_height = 224
img_width = 224

In [26]:
#Process data
preprocessor = keras.applications.resnet50.preprocess_input
data_generator = ImageDataGenerator(rescale=1./255, preprocessing_function=preprocessor, validation_split=0.2)


In [27]:
data_folder = '../images/uk_not_uk/train'
training_set = data_generator.flow_from_directory(data_folder, class_mode='binary',
                                                 target_size=(img_height,img_width),
                                                 subset="training", shuffle=True, seed=666,
                                                  batch_size=batch_size)
validation_set = data_generator.flow_from_directory(data_folder, class_mode='binary',
                                                    target_size=(img_height,img_width),
                                                    subset="validation", shuffle=True, seed=666,
                                                    batch_size=batch_size)

Found 48000 images belonging to 2 classes.
Found 12000 images belonging to 2 classes.


In [28]:
#Get pretrained model
resnet_model = tf.keras.applications.ResNet50(input_shape=(img_width, img_height, 3), include_top=False, pooling='avg', weights='imagenet')
resnet_model.trainable = False
resnet_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [29]:
#compile and set params
#model = keras.models.Sequential([resnet_model, keras.layers.Dense(1, name='logits')])
model = keras.models.Sequential([resnet_model, keras.layers.Dense(1, name='logits'), keras.layers.Activation('sigmoid',name='sigmoid_out')])
opt = keras.optimizers.Adam(learning_rate=0.005)
loss = keras.losses.BinaryCrossentropy()
checkpoint = ModelCheckpoint('./model_checkpoints/uk_not_uk/more_epochs_location_classifier', save_weights_only=True, monitor="val_loss", save_best_only=True)
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='binary_accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
    keras.metrics.Accuracy(name='accuracy'),
    keras.metrics.MeanAbsoluteError(name='mae')
]

model.compile(loss=loss, optimizer=opt, metrics=METRICS)

In [30]:
#load old model weights
model.load_weights('./model_checkpoints/uk_not_uk/location_classifier')

In [ ]:
#train
history = model.fit(training_set, epochs=40, validation_data=validation_set, callbacks=[checkpoint])

Epoch 1/40
750/750 [==============================] - 4092s 5s/step - loss: 0.4829 - tp: 8892.8549 - fp: 2487.2197 - tn: 9553.1531 - fn: 3130.6871 - binary_accuracy: 0.7662 - precision: 0.7799 - recall: 0.7407 - auc: 0.8501 - accuracy: 7.2718e-05 - mae: 0.3321 - val_loss: 0.4584 - val_tp: 4536.0000 - val_fp: 1065.0000 - val_tn: 4935.0000 - val_fn: 1464.0000 - val_binary_accuracy: 0.7893 - val_precision: 0.8099 - val_recall: 0.7560 - val_auc: 0.8682 - val_accuracy: 0.0000e+00 - val_mae: 0.3155
Epoch 2/40
750/750 [==============================] - 4064s 5s/step - loss: 0.4776 - tp: 8992.1944 - fp: 2469.6671 - tn: 9545.9454 - fn: 3056.1079 - binary_accuracy: 0.7708 - precision: 0.7841 - recall: 0.7473 - auc: 0.8536 - accuracy: 1.1854e-04 - mae: 0.3268 - val_loss: 0.5001 - val_tp: 5366.0000 - val_fp: 2391.0000 - val_tn: 3609.0000 - val_fn: 634.0000 - val_binary_accuracy: 0.7479 - val_precision: 0.6918 - val_recall: 0.8943 - val_auc: 0.8689 - val_accuracy: 0.0000e+00 - val_mae: 0.3317
Epoch

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



750/750 [==============================] - 4077s 5s/step - loss: 0.4639 - tp: 9077.7057 - fp: 2355.3196 - tn: 9674.1238 - fn: 2956.7656 - binary_accuracy: 0.7785 - precision: 0.7941 - recall: 0.7530 - auc: 0.8638 - accuracy: 2.8721e-04 - mae: 0.3177 - val_loss: 0.4646 - val_tp: 3971.0000 - val_fp: 583.0000 - val_tn: 5417.0000 - val_fn: 2029.0000 - val_binary_accuracy: 0.7823 - val_precision: 0.8720 - val_recall: 0.6618 - val_auc: 0.8733 - val_accuracy: 0.0000e+00 - val_mae: 0.3071
Epoch 8/40
750/750 [==============================] - 4077s 5s/step - loss: 0.4648 - tp: 9118.2530 - fp: 2363.1877 - tn: 9635.4194 - fn: 2947.0546 - binary_accuracy: 0.7795 - precision: 0.7947 - recall: 0.7570 - auc: 0.8631 - accuracy: 1.3380e-04 - mae: 0.3171 - val_loss: 0.4488 - val_tp: 4522.0000 - val_fp: 974.0000 - val_tn: 5026.0000 - val_fn: 1478.0000 - val_binary_accuracy: 0.7957 - val_precision: 0.8228 - val_recall: 0.7537 - val_auc: 0.8737 - val_accuracy: 0.0000e+00 - val_mae: 0.3051
Epoch 9/40
750/75

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



750/750 [==============================] - 4095s 5s/step - loss: 0.4328 - tp: 9395.1238 - fp: 2221.4727 - tn: 9779.5486 - fn: 2667.7696 - binary_accuracy: 0.7983 - precision: 0.8109 - recall: 0.7802 - auc: 0.8833 - accuracy: 1.2979e-04 - mae: 0.2940 - val_loss: 0.4403 - val_tp: 4275.0000 - val_fp: 653.0000 - val_tn: 5347.0000 - val_fn: 1725.0000 - val_binary_accuracy: 0.8018 - val_precision: 0.8675 - val_recall: 0.7125 - val_auc: 0.8834 - val_accuracy: 0.0000e+00 - val_mae: 0.2875
Epoch 27/40
714/750 [===========================>..] - ETA: 2:36 - loss: 0.4374 - tp: 8877.9104 - fp: 2111.9258 - tn: 9336.7843 - fn: 2553.3796 - binary_accuracy: 0.7967 - precision: 0.8091 - recall: 0.7767 - auc: 0.8803 - accuracy: 1.1877e-04 - mae: 0.2961

In [18]:
import pandas as pd
training_info = pd.DataFrame(history.history)
training_info.head()

,loss,tp,fp,tn,fn,binary_accuracy,precision,recall,auc,accuracy,...,val_tp,val_fp,val_tn,val_fn,val_binary_accuracy,val_precision,val_recall,val_auc,val_accuracy,val_mae
0,0.607360,14875.0,6966.0,17034.0,9125.0,0.664771,0.681059,0.619792,0.724288,0.000000,...,2781.0,415.0,5585.0,3219.0,0.697167,0.870150,0.463500,0.821266,0.0,0.383525
1,0.561719,16014.0,5983.0,18017.0,7986.0,0.708979,0.728008,0.667250,0.780652,0.000000,...,4184.0,1068.0,4932.0,1816.0,0.759667,0.796649,0.697333,0.837554,0.0,0.360385
2,0.540536,16487.0,5631.0,18369.0,7513.0,0.726167,0.745411,0.686958,0.802186,0.000000,...,4000.0,872.0,5128.0,2000.0,0.760667,0.821018,0.666667,0.847191,0.0,0.351955
3,0.524037,16791.0,5403.0,18597.0,7209.0,0.737250,0.756556,0.699625,0.817910,0.000000,...,4227.0,978.0,5022.0,1773.0,0.770750,0.812104,0.704500,0.853143,0.0,0.346838
4,0.516342,16986.0,5305.0,18695.0,7014.0,0.743354,0.762012,0.707750,0.824245,0.000021,...,4515.0,1196.0,4804.0,1485.0,0.776583,0.790580,0.752500,0.856080,0.0,0.339253


In [21]:
training_info.to_csv('./model_checkpoints/uk_not_uk/more_epochs_history.csv')

In [22]:
test_folder = '../images/uk_not_uk/test'
preprocessor = keras.applications.resnet50.preprocess_input
test_data_generator = ImageDataGenerator(rescale=1./255, preprocessing_function=preprocessor)
test_set = test_data_generator.flow_from_directory(test_folder, class_mode='binary',target_size=(img_height,img_width))

Found 16169 images belonging to 2 classes.


In [ ]:
model.load_weights('./model_checkpoints/uk_not_uk/location_classifier')

In [ ]:
res = model.evaluate(test_set)

496/506 [============================>.] - ETA: 22s - loss: 1.2298 - tp: 3602.0000 - fp: 55.0000 - tn: 128.0000 - fn: 12087.0000 - binary_accuracy: 0.2350 - precision: 0.9850 - recall: 0.2296 - auc: 0.4453 - accuracy: 0.0000e+00 - mae: 0.6401

In [32]:
import pandas as pd
hist = pd.read_csv(./model_checkpoints/uk_not_uk/location_classifier.csv)
hist.head(10)

SyntaxError: invalid syntax (<ipython-input-32-467240053c36>, line 2)